### Zigbang
- 동이름 입력하면 원룸 매물정보 가져오기
- !pip install geohash2
- 절차
    - 1. 동이름 > 위도, 경도
    - 2. 위도, 경도 > geohash
    - 3. geohash > item_ids
    - 4. item_ids > item datas

In [2]:
import geohash2

In [3]:
# 동이름 > 위도, 경도

addr = "신길동"
url = "https://apis.zigbang.com/v2/search?q={}&serviceType=원룸".format(addr)
response = requests.get(url)
data = response.json()["items"][1]
lat, lng = data["lat"], data["lng"]
lat, lng

In [ ]:
# 2. 위도, 경도 > geohash

In [16]:
geohash = geohash2.encode(lat, lng, 5)
geohash

'wydm2'

In [ ]:
# 3. geohash > item_ids

In [17]:
url = "https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={}\
&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸".format(geohash)
response = requests.get(url)
response

<Response [200]>

In [21]:
datas = response.json()["items"]
ids = [data["item_id"] for data in datas]
len(ids), ids[:3]

(1553, [24187857, 24385765, 24500582])

In [ ]:
# 4. item_ids > item datas

In [33]:
# 포스터 방식에서 쿼리문은 params로 추가해줌
params={
    "domain": "zigbang",
    "item_ids": ids[:100],
    "withCoalition": "true",
}
url = "https://apis.zigbang.com/v2/items/list"

response = requests.post(url, data=params)
response

<Response [200]>

In [34]:
items = response.json()["items"]
items[0]

{'section_type': None,
 'item_id': 24187857,
 'images_thumbnail': 'https://ic.zigbang.com/ic/items/24187857/1.jpg',
 'sales_type': '월세',
 'sales_title': '월세',
 'deposit': 200,
 'rent': 38,
 'size_m2': 23.14,
 '공급면적': {'m2': 23.14, 'p': '7'},
 '전용면적': {'m2': 23.14, 'p': '7'},
 '계약면적': None,
 'room_type_title': None,
 'floor': '3',
 'floor_string': '3',
 'building_floor': '5',
 'title': '남향/애견/단기/가능한 참 착한방 (English)',
 'is_first_movein': None,
 'room_type': '01',
 'address': '관악구 봉천동',
 'random_location': {'lat': 37.48511192926532, 'lng': 126.93509195966001},
 'is_zzim': False,
 'status': True,
 'service_type': '원룸',
 'tags': [],
 'address1': '서울시 관악구 봉천동',
 'address2': None,
 'address3': None,
 'manage_cost': '6',
 'reg_date': '2020-10-19T23:22:30+09:00',
 'is_new': False}

In [35]:
items_df = pd.DataFrame(items)
items_df.tail(4)

,address,address1,address2,address3,building_floor,deposit,floor,floor_string,images_thumbnail,is_first_movein,...,sales_type,section_type,service_type,size_m2,status,tags,title,계약면적,공급면적,전용면적
96,관악구 봉천동,서울시 관악구 봉천동,None,None,4,5000,1,1,https://ic.zigbang.com/ic/items/24553632/1.jpg,None,...,전세,None,원룸,21.49,True,[추천],🌞서울대입구역🌞저렴한풀옵션원룸전세🌞사이즈굳굳🌞,None,"{'m2': 21.49, 'p': '6.5'}","{'m2': 21.49, 'p': '6.5'}"
97,관악구 신림동,서울시 관악구 신림동,None,None,6,100,1,1,https://ic.zigbang.com/ic/items/24536115/1.jpg,None,...,월세,None,원룸,18.00,True,[추천],@초 저가 역세권원룸@ 이쁘고 깔끔하고 안전하기까지!!,None,"{'m2': 18, 'p': '5.4'}","{'m2': 18, 'p': '5.4'}"
98,관악구 신림동,서울시 관악구 신림동,None,None,8,7000,6,6,https://ic.zigbang.com/ic/items/24517748/1.jpg,None,...,월세,None,원룸,21.49,True,[추천],★무융자 건물★ -청년대출 가능-,None,"{'m2': 21.49, 'p': '6.5'}","{'m2': 21.49, 'p': '6.5'}"
99,관악구 신림동,서울시 관악구 신림동,None,None,10,8000,7,7,https://ic.zigbang.com/ic/items/24219502/1.jpg,None,...,월세,None,원룸,31.23,True,[추천],💕신림역 도보 2분!💕대출로 고민하지마시고 월세오세요!,None,"{'m2': 31.23, 'p': '9.4'}","{'m2': 31.23, 'p': '9.4'}"


In [ ]:
# 필요한 컬럼만 필터링 
# Pandas는 컬럼이 20개 넘어가면 ...으로 생략함

In [36]:
pd.options.display.max_columns = len(items_df)

In [37]:
items_df.tail(2)

,address,address1,address2,address3,building_floor,deposit,floor,floor_string,images_thumbnail,is_first_movein,is_new,is_zzim,item_id,manage_cost,random_location,reg_date,rent,room_type,room_type_title,sales_title,sales_type,section_type,service_type,size_m2,status,tags,title,계약면적,공급면적,전용면적
98,관악구 신림동,서울시 관악구 신림동,None,None,8,7000,6,6,https://ic.zigbang.com/ic/items/24517748/1.jpg,None,False,False,24517748,8,"{'lat': 37.486098529259436, 'lng': 126.9251824...",2020-10-27T15:44:48+09:00,10,01,None,월세,월세,None,원룸,21.49,True,[추천],★무융자 건물★ -청년대출 가능-,None,"{'m2': 21.49, 'p': '6.5'}","{'m2': 21.49, 'p': '6.5'}"
99,관악구 신림동,서울시 관악구 신림동,None,None,10,8000,7,7,https://ic.zigbang.com/ic/items/24219502/1.jpg,None,False,False,24219502,13,"{'lat': 37.485595223836725, 'lng': 126.9309417...",2020-10-05T21:13:47+09:00,20,02,None,월세,월세,None,원룸,31.23,True,[추천],💕신림역 도보 2분!💕대출로 고민하지마시고 월세오세요!,None,"{'m2': 31.23, 'p': '9.4'}","{'m2': 31.23, 'p': '9.4'}"


In [ ]:
# 5. 데이터 전처리

In [41]:
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2",
          "floor", "building_floor", "title", "address",
           "manage_cost", "random_location"]
result_df = item_df[columns]
result_df = result_df[result_df["address"].str.contains(addr)
result_df.reset_index(drop=True, inplace=True)

SyntaxError: invalid syntax (<ipython-input-41-2f1f85b5e19d>, line 6)

In [ ]:
result_df = result_df[result_df["addr"]]

In [39]:
# 함수로 만들기

In [40]:
def crawling_oneroom(addr):
    
    # 1. 동이름 > 위도, 경도
    url = "https://apis.zigbang.com/v2/search?q={}&serviceType=원룸".format(addr)
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    # 2. 위도, 경도 > geohash
    geohash = geohash2.encode(lat, lng, 5)
    
    # 3. geohash > item_ids
    url = "https://apis.zigbang.com/v2/items?\
deposit_gteq=0&domain=zigbang&geohash={}\
&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸".format(geohash)
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["item_id"] for data in datas]
    
    # 4. item_ids > item datas
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "item_ids": ids,
        "withCoalition": "true",
    }
    response = requests.post(url, data=params)
    items = response.json()["items"]
    item_df = pd.DataFrame(items)
    
    # 5. 데이터 전처리
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2",
          "floor", "building_floor", "title", "address",
           "manage_cost", "random_location"]
    result_df = item_df[columns]
    result_df = result_df[result_df["address"].str.contains(addr)]
    result_df.reset_index(drop=True, inplace=True)
    
    return result_df